# **Trader Performance vs Market Sentiment Analysis**

## 1. Objective

The objective of this analysis is to evaluate how Bitcoin market sentiment 
(Fear, Extreme Fear, Neutral, Greed, Extreme Greed) influences trader behavior 
and profitability on Hyperliquid.

The goal is to identify sentiment-driven behavioral patterns and derive 
actionable strategy recommendations.

## 2. Data Loading

We load the historical trader dataset and the Bitcoin Fear & Greed Index dataset.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

In [ ]:
trades = pd.read_csv("/kaggle/input/datasets/avijit18/historical-data/historical_data.csv")
sentiment = pd.read_csv("/kaggle/input/datasets/avijit18/fear-greed-index/fear_greed_index.csv")

In [ ]:
trades.columns = trades.columns.str.strip().str.lower()
sentiment.columns = sentiment.columns.str.strip().str.lower()

print(trades.head())
print(sentiment.head())

In [ ]:
print("Trades Shape:", trades.shape)
print("Sentiment Shape:", sentiment.shape)

print("\nMissing Values (Trades):\n", trades.isnull().sum())
print("\nMissing Values (Sentiment):\n", sentiment.isnull().sum())

print("\nDuplicate Rows (Trades):", trades.duplicated().sum())
print("Duplicate Rows (Sentiment):", sentiment.duplicated().sum())

## 3. Data Cleaning & Date Alignment

We standardize timestamps and align both datasets at daily level.

In [ ]:
# Convert trades timestamp (DD-MM-YYYY)
trades['timestamp ist'] = pd.to_datetime(
    trades['timestamp ist'],
    dayfirst=True,
    errors='coerce'
)

trades['date'] = trades['timestamp ist'].dt.date

# Convert sentiment date safely
sentiment['date'] = pd.to_datetime(
    sentiment['date'],
    errors='coerce'
).dt.date

# Merge
merged = trades.merge(sentiment, on='date', how='left')

print("Missing sentiment rows:",
      merged['classification'].isnull().sum())

## 4. Feature Engineering

We create:
- Win flag
- Daily PnL
- Trade frequency
- Trade size segmentation

In [ ]:
# Win Flag
merged['win'] = merged['closed pnl'] > 0

# Daily PnL
daily_pnl = (
    merged
    .groupby(['account','date'])['closed pnl']
    .sum()
    .reset_index()
)

# Trade Count
trade_count = (
    merged
    .groupby(['account','date'])
    .size()
    .reset_index(name='trade_count')
)

# Trade Size Segmentation
median_size = merged['size usd'].median()

merged['size_segment'] = np.where(
    merged['size usd'] > median_size,
    "Large Trade",
    "Small Trade"
)

In [ ]:
long_short = (
    merged
    .groupby(['date','side'])
    .size()
    .unstack(fill_value=0)
)

long_short['long_short_ratio'] = (
    long_short.get('long',0) /
    (long_short.get('short',0) + 1)
)

long_short.head()

## 5. Sentiment-Based Performance Analysis

We compare profitability, win rate, and volatility across sentiment regimes.

In [ ]:
# Performance Summary
performance = merged.groupby('classification')['closed pnl'].describe()
print(performance)

# Win Rate
win_rate = merged.groupby('classification')['win'].mean()
print(win_rate)

# Volatility
volatility = merged.groupby('classification')['closed pnl'].std()
print(volatility)

# Visualization
plt.figure()
sns.boxplot(x='classification', y='closed pnl', data=merged)
plt.title("PnL Distribution by Market Sentiment")
plt.xticks(rotation=45)
plt.show()

## 6. Trader Segmentation

We segment traders based on:
- Trade size
- Trading frequency
- Performance consistency

In [ ]:
# Size Segment Performance
size_perf = (
    merged
    .groupby(['classification','size_segment'])
    ['closed pnl']
    .mean()
)
print(size_perf)

# Frequent vs Infrequent
freq = merged.groupby('account').size()
median_freq = freq.median()

merged['freq_segment'] = merged['account'].apply(
    lambda x: "Frequent"
    if freq[x] > median_freq
    else "Infrequent"
)

freq_perf = (
    merged
    .groupby(['classification','freq_segment'])
    ['closed pnl']
    .mean()
)
print(freq_perf)

# Consistent Traders
consistency = merged.groupby('account')['win'].mean()

merged['consistency_segment'] = merged['account'].apply(
    lambda x: "Consistent"
    if consistency[x] > 0.6
    else "Inconsistent"
)

consistency_perf = (
    merged
    .groupby(['classification','consistency_segment'])
    ['closed pnl']
    .mean()
)
print(consistency_perf)

## 7. Key Insights & Strategy Recommendations

### Key Insights

1. Trader profitability is highest during Extreme Greed periods, showing the highest average PnL and win rate.

2. Fear and Extreme Fear regimes exhibit lower profitability and higher volatility.

3. Large trades consistently generate significantly higher average PnL across all sentiment regimes.

4. Sentiment clearly influences both trader performance and behavioral patterns.

---

### Strategy Recommendations

1. Increase exposure cautiously during Extreme Greed regimes where profitability and win rates are strongest.

2. Reduce trade size during Fear regimes to manage volatility risk.

3. Use sentiment classification as a dynamic risk-adjustment tool rather than a direct trading signal.

4. Apply tighter risk controls during sentiment transitions.